In [1]:
# Import needed moduls

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load data

df = pd.read_csv("data/train.csv")

X = df.drop("SalePrice", axis=1)
y = df.SalePrice
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [130]:
# Check missing values
df.isna().sum().sum()

6965

In [168]:
# Check for which numeric columns have null values
for label, content in X.items():
    
    if pd.api.types.is_string_dtype(content):
        X[label] = content.astype("category").cat.as_ordered()
        
for label, content in X.items():  
    X[label+"_is_missing"] = pd.isnull(content)
    if pd.api.types.is_numeric_dtype(content):
        
        if pd.isnull(content).sum():            
            # Fill missing numeric values with median since it's more robust than the mean
            X[label] = content.fillna(content.median())
            
    if not pd.api.types.is_numeric_dtype(content):
        # We add the +1 because pandas encodes missing categories as -1
        X[label] = pd.Categorical(content).codes+1       
df.isna().sum().sum()

C:\Users\rusla\AppData\Local\Temp\ipykernel_3500\2934352953.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[label+"_is_missing"] = pd.isnull(content)
C:\Users\rusla\AppData\Local\Temp\ipykernel_3500\2934352953.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[label+"_is_missing"] = pd.isnull(content)
C:\Users\rusla\AppData\Local\Temp\ipykernel_3500\2934352953.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

0

In [169]:
%%time
from sklearn.ensemble import RandomForestRegressor
# Instantiate model
model = RandomForestRegressor(n_jobs=-1)

# Fit the model
model.fit(X, y)

CPU times: total: 2.5 s
Wall time: 412 ms


RandomForestRegressor(n_jobs=-1)

In [172]:
model.score(X, y)

0.9812581027722013

In [173]:
from sklearn.metrics import mean_squared_log_error
def score_rmsle(model, X_test, y_test):
    return np.sqrt(mean_squared_log_error(model.predict(X_test), y_test))

In [174]:
score_rmsle(model, df.drop("SalePrice", axis=1), df.SalePrice)

0.05976619703843639

In [175]:
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [176]:
score_rmsle(model, X_train, y_train)

0.06236907354494306

In [179]:
%%time

rf_grid = {"n_estimators": np.arange(1, 102, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              rf_grid,
                              cv=5,
                              n_iter=50)

rs_model.fit(X, y)

CPU times: total: 1min 11s
Wall time: 1min 12s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=50,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([  1,  11,  21,  31,  41,  51,  61,  71,  81,  91, 101])})

In [180]:
rs_model.best_params_

{'n_estimators': 31,
 'min_samples_split': 14,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None}

In [181]:
rs_model.best_estimator_.score(X_test, y_test)

0.9622323500727641

In [182]:
score_rmsle(rs_model.best_estimator_, X_test, y_test)

0.08102550171402662

In [183]:
df_test = pd.read_csv("data/test.csv")


for label, content in df_test.items():
    
    if pd.api.types.is_string_dtype(content):
        df_test[label] = content.astype("category").cat.as_ordered()
        
for label, content in df_test.items():  
    df_test[label+"_is_missing"] = pd.isnull(content)
    if pd.api.types.is_numeric_dtype(content):
        
        if pd.isnull(content).sum():            
            # Fill missing numeric values with median since it's more robust than the mean
            df_test[label] = content.fillna(content.median())
            
    if not pd.api.types.is_numeric_dtype(content):
        # We add the +1 because pandas encodes missing categories as -1
        df_test[label] = pd.Categorical(content).codes+1      
        

C:\Users\rusla\AppData\Local\Temp\ipykernel_3500\869015042.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[label+"_is_missing"] = pd.isnull(content)
C:\Users\rusla\AppData\Local\Temp\ipykernel_3500\869015042.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[label+"_is_missing"] = pd.isnull(content)
C:\Users\rusla\AppData\Local\Temp\ipykernel_3500\869015042.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

In [186]:
len(df_test.columns) - len(X.columns)

0

In [187]:
a = rs_model.best_estimator_.predict(df_test)
a

array([128167.4921888 , 152318.61360004, 182396.22647038, ...,
       149243.12630755, 117803.98985307, 229481.50993459])

In [189]:
df_test["Id"]

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [194]:
pd.DataFrame({"Id": df_test["Id"],
              "SalePrice": rs_model.best_estimator_.predict(df_test)}).to_csv("data/first_preds.csv", index=False)


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [36]:
X[X.columns[X.isna().sum() > 0]].fillna(0)

,LotFrontage,Alley,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,0,BrkFace,196.0,Gd,TA,No,GLQ,Unf,SBrkr,0,Attchd,2003.0,RFn,TA,TA,0,0,0
1,80.0,0,None,0.0,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,1976.0,RFn,TA,TA,0,0,0
2,68.0,0,BrkFace,162.0,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,2001.0,RFn,TA,TA,0,0,0
3,60.0,0,None,0.0,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,1998.0,Unf,TA,TA,0,0,0
4,84.0,0,BrkFace,350.0,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,2000.0,RFn,TA,TA,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,0,None,0.0,Gd,TA,No,Unf,Unf,SBrkr,TA,Attchd,1999.0,RFn,TA,TA,0,0,0
1456,85.0,0,Stone,119.0,Gd,TA,No,ALQ,Rec,SBrkr,TA,Attchd,1978.0,Unf,TA,TA,0,MnPrv,0
1457,66.0,0,None,0.0,TA,Gd,No,GLQ,Unf,SBrkr,Gd,Attchd,1941.0,RFn,TA,TA,0,GdPrv,Shed
1458,68.0,0,None,0.0,TA,TA,Mn,GLQ,Rec,FuseA,0,Attchd,1950.0,Unf,TA,TA,0,0,0
